# To INSTALL packages

In [1]:
!pip install -U pip setuptools wheel
!pip install -U spacy

     |████████████████████████████████| 1.7 MB 8.4 MB/s 
     |████████████████████████████████| 958 kB 50.7 MB/s 
  Attempting uninstall: setuptools
    Found existing installation: setuptools 57.4.0
    Uninstalling setuptools-57.4.0:
      Successfully uninstalled setuptools-57.4.0
  Attempting uninstall: pip
    Found existing installation: pip 21.1.3
    Uninstalling pip-21.1.3:
      Successfully uninstalled pip-21.1.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


     |████████████████████████████████| 6.0 MB 8.4 MB/s            
     |████████████████████████████████| 451 kB 61.1 MB/s            
     |████████████████████████████████| 10.1 MB 57.5 MB/s            
     |████████████████████████████████| 628 kB 51.9 MB/s            
     |████████████████████████████████| 42 kB 1.3 MB/s             
     |████████████████████████████████| 181 kB 52.6 MB/s            
  Attempting uninstall: catalogue
    Found existing installation: catalogue 1.0.0
    Uninstalling catalogue-1.0.0:
      Successfully uninstalled catalogue-1.0.0
  Attempting uninstall: srsly
    Found existing installation: srsly 1.0.5
    Uninstalling srsly-1.0.5:
      Successfully uninstalled srsly-1.0.5
  Attempting uninstall: thinc
    Found existing installation: thinc 7.4.0
    Uninstalling thinc-7.4.0:
      Successfully uninstalled thinc-7.4.0
  Attempting uninstall: spacy
    Found existing installation: spacy 2.2.4
    Uninstalling spacy-2.2.4:
      Successfully uni

In [2]:
!pip install -U pip setuptools wheel
!pip install -U spacy
!python -m spacy download en_core_web_sm
!python -m spacy download pt_core_news_sm

     |████████████████████████████████| 13.9 MB 8.2 MB/s            
  Attempting uninstall: en-core-web-sm
    Found existing installation: en-core-web-sm 2.2.5
    Uninstalling en-core-web-sm-2.2.5:
      Successfully uninstalled en-core-web-sm-2.2.5
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
     |████████████████████████████████| 22.2 MB 1.3 MB/s            
✔ Download and installation successful
You can now load the package via spacy.load('pt_core_news_sm')


# To READ files

In [3]:
# Google drive conection
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### List files

In [4]:
path = "/content/drive/MyDrive/INSPER/03_trabfinal_1certif/Dados/Processed/"

from os import listdir
files = listdir(path)
files

['1721034292748976_Isac Godinho_2022-01-06_Folhajus.txt',
 '1721167833180765_Wesley Faraó Klimpel_2022-01-05_Mobilidade Urbana.txt',
 '1721155128269049_Isabella Menon_2022-01-05_Coronavírus.txt',
 '1721153153464175_José Matheus Santos_2022-01-05_Coronavírus.txt',
 '1721122587219657_Luís Costa_2022-01-05_Rio de Janeiro.txt',
 '1721134973363736_João Pedro Pitombo_2022-01-05_Coronavírus.txt',
 '1721130095679618_Fábio Pescarini_2022-01-05_Coronavírus.txt']

In [5]:
file1 = open(path + files[5], "r+")

text = file1.read()
text

'Lavagem do Bonfim é cancelada pelo 2º ano consecutivo em Salvador. Tradição há 249 anos, cortejo de culto a Oxalá e Senhor do Bonfim costuma reunir um milhão. A Prefeitura de Salvador confirmou nesta quarta-feira (5) o cancelamento da Lavagem do Bonfim, festa religiosa que costuma reunir cerca de um milhão de pessoas nas ruas da cidade.    Em uma rede social, o prefeito Bruno Reis (DEM) justificou a medida afirmando que houve aumento dos casos de Covid-19 e de gripe na capital.    "Não podemos dar margem para ampliação dos números de casos de infecção em nossa cidade. Precisamos ter a consciência e a responsabilidade na preservação das vidas", afirmou.    O Carnaval de rua de Salvador já havia sido cancelado pela prefeitura e governo do estado nas últimas semanas. A tendência é que as demais festas populares, com a Festa de Iemanjá e Lavagem de Itapuã também não aconteçam em 2022.    Festas privadas com até 5.000 pessoas, contudo, estão mantidas, inclusive no período carnavalesco e da

### Sentence Tokenization (by ' "(double quote)' and '.')

In [6]:
paragraphs = text.split('    ')
paragraphs

['Lavagem do Bonfim é cancelada pelo 2º ano consecutivo em Salvador. Tradição há 249 anos, cortejo de culto a Oxalá e Senhor do Bonfim costuma reunir um milhão. A Prefeitura de Salvador confirmou nesta quarta-feira (5) o cancelamento da Lavagem do Bonfim, festa religiosa que costuma reunir cerca de um milhão de pessoas nas ruas da cidade.',
 'Em uma rede social, o prefeito Bruno Reis (DEM) justificou a medida afirmando que houve aumento dos casos de Covid-19 e de gripe na capital.',
 '"Não podemos dar margem para ampliação dos números de casos de infecção em nossa cidade. Precisamos ter a consciência e a responsabilidade na preservação das vidas", afirmou.',
 'O Carnaval de rua de Salvador já havia sido cancelado pela prefeitura e governo do estado nas últimas semanas. A tendência é que as demais festas populares, com a Festa de Iemanjá e Lavagem de Itapuã também não aconteçam em 2022.',
 'Festas privadas com até 5.000 pessoas, contudo, estão mantidas, inclusive no período carnavalesco

In [24]:
str_in_doublequotes = r'"[^"]*\."|("[^"]*")*([^".]*\.)'

sentences = []

import re

for pg in paragraphs:
  # Collects the quotes inside the paragraph 
  in_quotes = re.findall(str_in_doublequotes, pg)
  #print(in_quotes)
  for grp in in_quotes:
    aux = "".join(grp)
    aux_trim = aux.strip()
    sentences.append(aux_trim)
    #print(aux_trim)

In [25]:
sentences

['Lavagem do Bonfim é cancelada pelo 2º ano consecutivo em Salvador.',
 'Tradição há 249 anos, cortejo de culto a Oxalá e Senhor do Bonfim costuma reunir um milhão.',
 'A Prefeitura de Salvador confirmou nesta quarta-feira (5) o cancelamento da Lavagem do Bonfim, festa religiosa que costuma reunir cerca de um milhão de pessoas nas ruas da cidade.',
 'Em uma rede social, o prefeito Bruno Reis (DEM) justificou a medida afirmando que houve aumento dos casos de Covid-19 e de gripe na capital.',
 '"Não podemos dar margem para ampliação dos números de casos de infecção em nossa cidade. Precisamos ter a consciência e a responsabilidade na preservação das vidas", afirmou.',
 'O Carnaval de rua de Salvador já havia sido cancelado pela prefeitura e governo do estado nas últimas semanas.',
 'A tendência é que as demais festas populares, com a Festa de Iemanjá e Lavagem de Itapuã também não aconteçam em 2022.',
 'Festas privadas com até 5.',
 '000 pessoas, contudo, estão mantidas, inclusive no per

In [ ]:
#import nltk
#nltk.download('punkt')

In [21]:
#from nltk.tokenize import sent_tokenize
## Split sentences by '.'  
#sentences2 = sent_tokenize(text)
#sentences2

['Lavagem do Bonfim é cancelada pelo 2º ano consecutivo em Salvador.',
 'Tradição há 249 anos, cortejo de culto a Oxalá e Senhor do Bonfim costuma reunir um milhão.',
 'A Prefeitura de Salvador confirmou nesta quarta-feira (5) o cancelamento da Lavagem do Bonfim, festa religiosa que costuma reunir cerca de um milhão de pessoas nas ruas da cidade.',
 'Em uma rede social, o prefeito Bruno Reis (DEM) justificou a medida afirmando que houve aumento dos casos de Covid-19 e de gripe na capital.',
 '"Não podemos dar margem para ampliação dos números de casos de infecção em nossa cidade.',
 'Precisamos ter a consciência e a responsabilidade na preservação das vidas", afirmou.',
 'O Carnaval de rua de Salvador já havia sido cancelado pela prefeitura e governo do estado nas últimas semanas.',
 'A tendência é que as demais festas populares, com a Festa de Iemanjá e Lavagem de Itapuã também não aconteçam em 2022.',
 'Festas privadas com até 5.000 pessoas, contudo, estão mantidas, inclusive no perí

### Recognize sentences with quotes

In [10]:
# This function returns the number of quotes found at a sentence
def regex(string):
    pattern1 = re.compile(r'"{1}') 
    result1 = re.search(r'"{1}', string)

    result2 = re.search(r'["][\w\s]+["]', string)

    if result1:
      return 1
    elif result2:
      return 2
    return 0


sent_numb_quotes = [regex(sent) for sent in sentences]
sent_numb_quotes

[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [11]:
# To return the index of the sentence list that have at least one quote

def condition_x_gt_0(x): return x > 0
index_quote = [idx for idx, element in enumerate(sent_numb_quotes) if condition_x_gt_0(element)]
index_quote

[4]

### Apply trained entity recognition model

In [12]:
# Load trained Entity Recognition model in Portuguese based on the references:

# https://monkeylearn.com/blog/named-entity-recognition/
# https://spacy.io/usage/linguistic-features#tokenization

import spacy
nlp = spacy.load("pt_core_news_sm")

In [13]:
# Apply entity recognition model by sentence & Keep only the PERSON Entity entries 

entities = []
for sent in sentences:
  sent_ent = nlp(sent)
  # Entity recognition
  ents = [(e.text, e.label_) for e in sent_ent.ents if e.label_ == "PER"]
  entities.append(ents)

In [14]:
entities

[[],
 [],
 [],
 [('Bruno Reis', 'PER')],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [('Dali', 'PER')],
 [],
 [('Theodósio Rodrigues de Faria', 'PER')],
 [],
 [],
 []]

In [26]:
# To return a vector that has 1 at the entry that has an entity recognized and 0 otherwise

def empty_part(part):
  if part:
      return 1
  return 0

sent_entity = [empty_part(part) for part in entities]
sent_entity

[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0]

In [27]:
# Which sentences have " (double quote) ?
# To return the index of the sentence list that has at least one entity recognized

index_ent = [idx for idx, element in enumerate(sent_entity) if condition_x_gt_0(element)]
index_ent

[3, 16, 18]

In [28]:
# All possible pair combination of 'sentence with quotation index' and 'sentence with entity index'

pair_idquote_ident = [(x, y) for x in index_quote for y in index_ent]
pair_idquote_ident

[(4, 3), (4, 16), (4, 18)]

In [29]:
# Difference between each pair of sentence indexes

def dif_pair(pair): return abs(pair[0] - pair[1])

dif_pair_idquote_ident = [dif_pair(pair) for pair in pair_idquote_ident]
dif_pair_idquote_ident

[1, 12, 14]

In [30]:
# Convert 'pair_idquote_ident' into numpy array
import numpy as np

pair_idquote_ident_array = np.asarray(pair_idquote_ident)
pair_idquote_ident_array

array([[ 4,  3],
       [ 4, 16],
       [ 4, 18]])

In [31]:
# Convert 'dif_pair_idquote_ident' into numpy array

dif_pair_idquote_ident_array = np.array(dif_pair_idquote_ident)

In [32]:
# 'cbind' pair_idquote_ident_array and dif_pair_idquote_ident_array

dif_pair_idquote_ident_dif_array = np.column_stack((pair_idquote_ident_array, 
                                                    dif_pair_idquote_ident_array))

In [33]:
# Convert dif_pair_idquote_ident_dif_array into a long format data.frame

import pandas as pd

dif_pair_idquote_ident_df = pd.DataFrame(dif_pair_idquote_ident_dif_array,
    columns = ['quote', 'entity', 'dif'])
dif_pair_idquote_ident_df

,quote,entity,dif
0,4,3,1
1,4,16,12
2,4,18,14


In [34]:
# Pivot long format object to wide format

dif_pair_idquote_ident_df_pv = dif_pair_idquote_ident_df.pivot(index = 'quote', columns = 'entity', values = 'dif')
dif_pair_idquote_ident_df_pv

entity,3,16,18
quote,,,
4,1,12,14


In [35]:
# Column index associated with the minimum value by row 

pos = dif_pair_idquote_ident_df_pv.apply(np.argmin, axis = 'columns')
pos

quote
4    0
dtype: int64

In [36]:
list(dif_pair_idquote_ident_df_pv.columns[pos])

[3]

In [37]:
entities

[[],
 [],
 [],
 [('Bruno Reis', 'PER')],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [('Dali', 'PER')],
 [],
 [('Theodósio Rodrigues de Faria', 'PER')],
 [],
 [],
 []]

In [57]:
from itertools import chain

entities_list = list(chain.from_iterable(entities))
entities_list

[('Bruno Reis', 'PER'),
 ('Dali', 'PER'),
 ('Theodósio Rodrigues de Faria', 'PER')]

In [61]:
df_entities = pd.DataFrame(entities_list, columns = ['entity', 'type'])

# Filtering Entities that have at least one capital letter
df_entities = df_entities[df_entities['entity'].str.match("[A-Z]+") == True]

# Split the name into First and Family Name
df_entities[['FirstName', 'FamilyName']] = df_entities['entity'].str.split(' ', 1, expand = True)

# First name in Capital letter
df_entities['FirstName'] = df_entities['FirstName'].str.upper()

# Remove accents from the First Name
df_entities['FirstName'] = df_entities['FirstName'].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')

df_entities

,entity,type,FirstName,FamilyName
0,Bruno Reis,PER,BRUNO,Reis
1,Dali,PER,DALI,None
2,Theodósio Rodrigues de Faria,PER,THEODOSIO,Rodrigues de Faria


In [39]:
#a_list = entities
indices_to_access = list(dif_pair_idquote_ident_df_pv.columns[pos])

entities_series = pd.Series(entities)
accessed_series = entities_series[indices_to_access]
accessed_list = list(accessed_series)
accessed_list = [i[0][0] for i in accessed_list] # Inspired by: https://stackoverflow.com/questions/12142133/how-to-get-first-element-in-a-list-of-tuples
print(accessed_list) # [0][0][0]

['Bruno Reis']


In [47]:
d = {'quote': dif_pair_idquote_ident_df_pv.index, 
     'entity_index': indices_to_access,
     'entity': accessed_list
     }
df_quote = pd.DataFrame(data = d)
df_quote

,quote,entity_index,entity
0,4,3,Bruno Reis


In [48]:
# Filtering Entities that have at least one capital letter
df_quote = df_quote[df_quote['entity'].str.match("[A-Z]+") == True]
df_quote

,quote,entity_index,entity
0,4,3,Bruno Reis


In [49]:
# Split the name into First and Family Name
df_quote[['FirstName', 'FamilyName']] = df_quote['entity'].str.split(' ', 1, expand = True)

# First name in Capital letter
df_quote['FirstName'] = df_quote['FirstName'].str.upper()

# Remove accents from the First Name
df_quote['FirstName'] = df_quote['FirstName'].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')

df_quote

,quote,entity_index,entity,FirstName,FamilyName
0,4,3,Bruno Reis,BRUNO,Reis


## Gender assignment

In [50]:
# https://brasil.io/dataset/genero-nomes/nomes/

path = "/content/drive/MyDrive/INSPER/03_trabfinal_1certif/Dados/"

names = pd.read_csv(path + 'nomes.csv')  
names_aux = names[['first_name', 'classification']]
names_aux

,first_name,classification
0,AALINE,F
1,AARAO,M
2,AARON,M
3,ABA,F
4,ABADE,M
...,...,...
100782,MACLEIA,F
100783,GIULINE,F
100784,DEMAILTON,M
100785,ILIVIA,F


In [51]:
df_quote = df_quote.merge(names_aux, left_on = 'FirstName', right_on = 'first_name')
df_quote

,quote,entity_index,entity,FirstName,FamilyName,first_name,classification
0,4,3,Bruno Reis,BRUNO,Reis,BRUNO,M


In [63]:
df_entities = df_entities.merge(names_aux, left_on = 'FirstName', right_on = 'first_name')
df_entities

,entity,type,FirstName,FamilyName,first_name_x,classification_x,first_name_y,classification_y
0,Bruno Reis,PER,BRUNO,Reis,BRUNO,M,BRUNO,M
1,Dali,PER,DALI,None,DALI,F,DALI,F
2,Theodósio Rodrigues de Faria,PER,THEODOSIO,Rodrigues de Faria,THEODOSIO,M,THEODOSIO,M
